In [1]:
import pandas as pd

In [2]:
from fyers_apiv3 import fyersModel
import pandas as pd
import os
import pandas as pd
import plotly.graph_objects as go
import datetime as dt
from datetime import timedelta
from fyers_apiv3.FyersWebsocket import data_ws
with open("abcd.txt", 'r') as r:
    access_token = r.read()
client_id = "DZO41L3M36-100"
fyers = fyersModel.FyersModel(
    client_id=client_id, is_async=False, token=access_token, log_path=os.getcwd())
fyers.get_profile()

{'s': 'ok',
 'code': 200,
 'message': '',
 'data': {'fy_id': 'YL00137',
  'name': 'LOKESH TALLURI',
  'image': None,
  'display_name': None,
  'pin_change_date': '22-07-2024 22:41:12',
  'email_id': 'tallurilokesh2003@gmail.com',
  'pwd_change_date': None,
  'PAN': '---------',
  'mobile_number': '9515652294',
  'totp': True,
  'pwd_to_expire': 90}}

In [3]:
lines = [
    "NSE:NIFTY50-INDEX",
    "NSE:NIFTYBANK-INDEX",
    "NSE:ASIANPAINT-EQ",
    "NSE:EICHERMOT-EQ",
    "NSE:HEROMOTOCO-EQ",
    "NSE:ITC-EQ",
    "NSE:RELIANCE-EQ",
    "NSE:TATASTEEL-EQ",
    "NSE:DRREDDY-EQ",
    "NSE:BPCL-EQ",
    "NSE:INFY-EQ",
    "NSE:BRITANNIA-EQ",
    "NSE:NESTLEIND-EQ",
    "NSE:HINDALCO-EQ",
    "NSE:LT-EQ",
    "NSE:TATACONSUM-EQ",
    "NSE:WIPRO-EQ",
    "NSE:TITAN-EQ",
    "NSE:KOTAKBANK-EQ",
    "NSE:BAJFINANCE-EQ",
    "NSE:CIPLA-EQ",
    "NSE:GRASIM-EQ",
    "NSE:HINDUNILVR-EQ",
    "NSE:M&M-EQ",
    "NSE:TATAMOTORS-EQ",
    "NSE:APOLLOHOSP-EQ",
    "NSE:SBIN-EQ",
    "NSE:UPL-EQ",
    "NSE:ADANIENT-EQ",
    "NSE:SUNPHARMA-EQ",
    "NSE:JSWSTEEL-EQ",
    "NSE:HDFCBANK-EQ",
    "NSE:TCS-EQ",
    "NSE:ICICIBANK-EQ",
    "NSE:POWERGRID-EQ",
    "NSE:MARUTI-EQ",
    "NSE:INDUSINDBK-EQ",
    "NSE:AXISBANK-EQ",
    "NSE:HCLTECH-EQ",
    "NSE:ONGC-EQ",
    "NSE:NTPC-EQ",
    "NSE:COALINDIA-EQ",
    "NSE:BHARTIARTL-EQ",
    "NSE:TECHM-EQ",
    "NSE:LTIM-EQ",
    "NSE:DIVISLAB-EQ",
    "NSE:ADANIPORTS-EQ",
    "NSE:HDFCLIFE-EQ",
    "NSE:SBILIFE-EQ",
    "NSE:ULTRACEMCO-EQ",
    "NSE:BAJAJ-AUTO-EQ",
    "NSE:BAJAJFINSV-EQ"
]

In [4]:
import mysql.connector

# Define database connection details
db_config = {
    'host': '118.139.182.3',
    'user': 'sqluser1',
    'password': 'TGDp0U&[1Y4S',
    'database': 'stocks'
}


con = mysql.connector.connect(**db_config)
cur = con.cursor()

for line in lines:
    line = line.strip()
    data = {
            "symbol": line,
            "resolution": "5",
            "date_format": "1",
            "range_from": "2024-08-01",
            "range_to": "2024-08-01",
            "cont_flag": "1"
        }
    df = fyers.history(data=data)
    df = pd.DataFrame(df['candles'])
    df.columns = ['date', 'open', 'high', 'low', 'close', 'volume']
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.date = (df.date.dt.tz_localize('UTC').dt.tz_convert('Asia/Kolkata'))
    df['date'] = df['date'].dt.tz_localize(None)
    df_sorted = df.sort_values(by=['date'], ascending=True)
    df = df_sorted.drop_duplicates(subset='date', keep='first')
    df.reset_index(drop=True, inplace=True)

    parts = line.split(':')[-1].replace('-', '_').replace('&', '_')
    line = parts.lower()

    for _, row in df.iterrows():
        query = f"""
                INSERT INTO {line} VALUES ('{row['date']}',{row['open']},{row['high']},{row['low']},{row['close']},{row['volume']})
                """
#     query = f"""
#                 DELETE FROM {line} WHERE `date` > '2024-06-13'
#                 """
        cur.execute(query)
    con.commit()
con.close()
